# Classification

## Word Preprocessing

#### 斷詞 & 寫入檔案

In [ ]:
import pandas as pd

In [ ]:
!pip install monpa

In [ ]:
import csv
import monpa
from monpa import utils
from collections import Counter

down_tf_counter = Counter()
down_df_counter = Counter()

result=list() #預備儲存處理過的結果
f = open('limit_down.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
i=0
for row in rows: #對內容逐行列舉，row[0]是title, row[1]是content
  i=i+1 

  if(i>1): #第一行是欄位名稱，略過
        
    #以下套用斷詞小範例
    query=row[0]+' '+row[1]
    df_tmp = Counter() #暫存本篇 df 用
    str='' #暫存本篇斷詞後的內容用
    sentence_list = utils.short_sentence(query) #斷句
    for item in sentence_list:
      result_cut = monpa.cut(item) #斷詞
      for term in result_cut:
        term=term.strip() #去除前後多餘空白
        if(len(term)>1): #若詞長>1
           str=str+' '+term
           down_tf_counter[term] += 1
           if df_tmp[term] == 0:
             df_tmp[term] = 1
    down_df_counter += df_tmp
    # print(i, str)
    result.append(str)

f.close() #關閉檔案

In [ ]:
for term in down_tf_counter.most_common(100): #印出tf前100名
  print(term[0],term[1])

In [ ]:
f = open('limit_down_ok.csv', 'w', newline='', encoding='utf-8') #以寫入模式開啟另一個檔
writer = csv.writer(f) #準備寫入
for str in result: #對結果逐行列舉
  writer.writerow([str]) #寫入一行
f.close()

In [ ]:
up_tf_counter = Counter()
up_df_counter = Counter()

result=list() #預備儲存處理過的結果
f = open('limit_up.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
i=0
for row in rows: #對內容逐行列舉，row[0]是title, row[1]是content
  i=i+1 

  if(i>1): #第一行是欄位名稱，略過
        
    #以下套用斷詞小範例
    query=row[0]+' '+row[1]
    df_tmp = Counter() #暫存本篇 df 用
    str='' #暫存本篇斷詞後的內容用
    sentence_list = utils.short_sentence(query) #斷句
    for item in sentence_list:
      result_cut = monpa.cut(item) #斷詞
      for term in result_cut:
        term=term.strip() #去除前後多餘空白
        if(len(term)>1): #若詞長>1
           str=str+' '+term
           up_tf_counter[term] += 1
           if df_tmp[term] == 0:
             df_tmp[term] = 1
    up_df_counter += df_tmp
    #print(i, str)
    result.append(str)

f.close() #關閉檔案

In [ ]:
for term in up_tf_counter.most_common(100): #印出tf前100名
  print(term[0],term[1])

In [ ]:
f = open('limit_up_ok.csv', 'w', newline='', encoding='utf-8') #以寫入模式開啟另一個檔
writer = csv.writer(f) #準備寫入
for str in result: #對結果逐行列舉
  writer.writerow([str]) #寫入一行
f.close()

#### 編列字典

In [ ]:
f = open('limit_down_term.csv', 'w', newline='', encoding='utf-8') #以寫入模式開啟另一個檔
writer = csv.writer(f) #準備寫入
for term in down_tf_counter.most_common(100): #印出tf前100名
  writer.writerow(term) #寫入一行
f.close()

In [ ]:
f = open('limit_up_term.csv', 'w', newline='', encoding='utf-8') #以寫入模式開啟另一個檔
writer = csv.writer(f) #準備寫入
for term in up_tf_counter.most_common(100): #印出tf前100名
  writer.writerow(term) #寫入一行
f.close()

#### 建構 VSM

In [ ]:
import csv

content=list() #用串列預備儲存所有內容

f = open('limit_up_ok.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows: 
  content.append(row[0])
f.close()
print(len(content))

f = open('limit_down_ok.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  content.append(row[0])
f.close()
print(len(content))

612
738


In [ ]:
termset=dict() #用字典預備儲存所有詞彙
no=0 #幫詞彙編號

f = open('limit_up_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

f = open('limit_down_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

In [ ]:
print(len(termset)) #檢查總共有幾個詞彙

145


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=termset,use_idf=True) #用給定的termset建構向量，並使用idf加權

## Classification with KNN

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

X = vectorizer.fit_transform(content) #用給定的文件集做為input
Y = [1]*612+[0]*126 # 此時X的前612行是來自limit_up.csv, 後126行來自limit_down.csv，令Up(漲停)label為1，Down(跌停)label為0。

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, stratify=Y) #將Dataset做shuffle並取10%的data作為testing data

knn_model = KNeighborsClassifier(n_neighbors=3) # 設定KNN的近鄰參數
knn_model.fit(x_train,y_train) #用training set fit KNN model

train_preds = knn_model.predict(x_train) #分別在training set和testing set上做預測
test_preds = knn_model.predict(x_test)

expected_results = []
predicted_results = []

expected_results.extend(y_train)
predicted_results.extend(train_preds)
print('===================Training Dataset==================')
print(metrics.classification_report(expected_results, predicted_results))

expected_results = []
predicted_results = []

expected_results.extend(y_test)
predicted_results.extend(test_preds)
print('===================Validation Dataset===================')
print(metrics.classification_report(expected_results, predicted_results))

===================Training Dataset==================
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       113
           1       0.96      0.97      0.96       551

    accuracy                           0.93       664
   macro avg       0.89      0.87      0.88       664
weighted avg       0.93      0.93      0.93       664

===================Testing Dataset===================
              precision    recall  f1-score   support

           0       0.90      0.69      0.78        13
           1       0.94      0.98      0.96        61

    accuracy                           0.93        74
   macro avg       0.92      0.84      0.87        74
weighted avg       0.93      0.93      0.93        74



#### 給定一段文字，判斷是漲停還跌停

In [ ]:
import monpa
from monpa import utils
query='倫敦鎳價又起波瀾，週四(3月24日)連續第二個交易日飆上漲停板(15%)，帶動上海鎳價週五也飆漲停板。市場傳出，有「鎳王」之稱的青山集團董事長項光達最近已回補了部分空單。\
  彭博社24日引述未具名消息人士報導，項光達本週趁鎳市暫時解封、透過倫敦金屬交易所(LME)買進鎳期貨，藉以回補空單。\
  不過，項光達及同伴僅回補數萬噸的空方部位，手上空單仍龐大。據消息，鎳市3月8日暫停交易時，項光達握有超過150,000噸空方部位，他旗下事業及交易夥伴也持有大批空單。'
str='' #暫存本篇斷詞後的內容用
sentence_list = utils.short_sentence(query) #斷句
for item in sentence_list:
  result_cut = monpa.cut(item) #斷詞
  for term in result_cut:
    term=term.strip() #去除前後多餘空白
    if(len(term)>1): #若詞長>1
      str=str+' '+term
print(str)
q=vectorizer.fit_transform([str]) #將查詢也套入同一個向量空間 

In [ ]:
query_pred = knn_model.predict(q)
print("this query belongs to", "漲停" if query_pred[0] == 1 else "跌停")

this query belongs to 漲停
